In [6]:
import pandas as pd
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import classification_report, plot_confusion_matrix,fbeta_score, make_scorer
import seaborn as sns
from sklearn.model_selection import StratifiedKFold


import time

In [12]:
time_begin = time.time()
data = pd.read_csv("wine_quality.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed.acidity         6497 non-null   float64
 1   volatile.acidity      6497 non-null   float64
 2   citric.acid           6497 non-null   float64
 3   residual.sugar        6497 non-null   float64
 4   chlorides             6497 non-null   float64
 5   free.sulfur.dioxide   6497 non-null   float64
 6   total.sulfur.dioxide  6497 non-null   float64
 7   density               6497 non-null   float64
 8   pH                    6497 non-null   float64
 9   sulphates             6497 non-null   float64
 10  alcohol               6497 non-null   float64
 11  quality               6497 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 609.2 KB


## Divisão de base de dados e separação de base

In [13]:
#X = data[["fixed.acidity","volatile.acidity","citric.acid","residual.sugar","chlorides","free.sulfur.dioxide","total.sulfur.dioxide","density","pH","sulphates","alcohol"]]
#y = data[["quality"]]
X = data.iloc[:, 0:11].values
y = data.iloc[:, 11].values

In [14]:
def print_distribution(arr, print_nl=True):
    arr = np.unique(arr, return_counts=True)[1] / len(arr)
    for i in range(arr.shape[0]):
        print("Classe %d: %.2f%%" %(i, arr[i]*100))

    if print_nl:
        print("\n")

In [15]:
print("Proporções por classe no dataset em geral:")
print_distribution(y)

kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=199)
fold = 0
for train, test in kf.split(X, y):
    print("Fold %d" %(fold))
    print_distribution(y[train], print_nl=(fold != 2))
    fold += 1

Proporções por classe no dataset em geral:
Classe 0: 0.46%
Classe 1: 3.32%
Classe 2: 32.91%
Classe 3: 43.65%
Classe 4: 16.61%
Classe 5: 2.97%
Classe 6: 0.08%


Fold 0
Classe 0: 0.46%
Classe 1: 3.34%
Classe 2: 32.91%
Classe 3: 43.65%
Classe 4: 16.61%
Classe 5: 2.96%
Classe 6: 0.09%


Fold 1
Classe 0: 0.46%
Classe 1: 3.34%
Classe 2: 32.91%
Classe 3: 43.65%
Classe 4: 16.61%
Classe 5: 2.96%
Classe 6: 0.09%


Fold 2
Classe 0: 0.46%
Classe 1: 3.34%
Classe 2: 32.91%
Classe 3: 43.65%
Classe 4: 16.61%
Classe 5: 2.98%
Classe 6: 0.07%
Fold 3
Classe 0: 0.46%
Classe 1: 3.32%
Classe 2: 32.91%
Classe 3: 43.65%
Classe 4: 16.62%
Classe 5: 2.98%
Classe 6: 0.07%


Fold 4
Classe 0: 0.46%
Classe 1: 3.32%
Classe 2: 32.91%
Classe 3: 43.66%
Classe 4: 16.61%
Classe 5: 2.98%
Classe 6: 0.07%


Fold 5
Classe 0: 0.46%
Classe 1: 3.32%
Classe 2: 32.91%
Classe 3: 43.66%
Classe 4: 16.61%
Classe 5: 2.98%
Classe 6: 0.07%


Fold 6
Classe 0: 0.46%
Classe 1: 3.32%
Classe 2: 32.91%
Classe 3: 43.66%
Classe 4: 16.61%
Classe 5

C:\Users\cfpc2\anaconda3\envs\rv\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 5 members, which is less than n_splits=10.
  warnings.warn(


In [16]:
kf_results = list()

from sklearn.metrics import make_scorer, f1_score,precision_score,recall_score
scores = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score, average = 'macro'),
           'recall': make_scorer(recall_score, average = 'macro'),
           'f1_macro': make_scorer(f1_score, average = 'macro'),
           'f1_weighted': make_scorer(f1_score, average = 'weighted')}

In [17]:
def createANN(epochs, activation, neurons):
    classifier = Sequential()
    classifier.add(Dense(units=neurons, activation=activation,input_dim=11))
    classifier.add(Dropout(0.1))
    classifier.add(Dense(units=neurons, activation=activation))
    classifier.add(Dropout(0.1))
    classifier.add(Dense(units=10, activation="softmax"))
    classifier.compile(optimizer="adam", loss="sparse_categorical_crossentropy",
                       metrics=["accuracy"])
    classifier.fit(X_train, y_train, batch_size=16,
                      epochs=epochs)
    return classifier

In [18]:
for train_ix, test_ix in kf.split(X,y):
    X_train, X_test = X[train_ix, :], X[test_ix, :]
    y_train, y_test = y[train_ix], y[test_ix]
    #cv_inner = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
    classifier = KerasClassifier(build_fn=createANN)
    parameters={"epochs": [10,20],
                "activation": ["linear", "relu"],
                "neurons": [4,8]}
    # define search space
    search = GridSearchCV(estimator=classifier, param_grid=parameters,
                          scoring="accuracy", cv=4, refit=True)
    # execute search
    result = search.fit(X_train, y_train)
    # get the best performing model fit on the whole training set
    best_model = result.best_estimator_
    # evaluate model on the hold out dataset
    yhat = best_model.predict(X_test)
    # evaluate the model
    acc = accuracy_score(y_test, yhat)
    # store the result
    kf_results.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_params_))

C:\Users\cfpc2\anaconda3\envs\rv\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 5 members, which is less than n_splits=10.
  warnings.warn(
C:\Users\cfpc2\AppData\Local\Temp/ipykernel_23948/4205237341.py:5: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  classifier = KerasClassifier(build_fn=createANN)


Epoch 1/10
366/366 [==============================] - 1s 1ms/step - loss: 3.5458 - accuracy: 0.3215
Epoch 2/10
366/366 [==============================] - 0s 1ms/step - loss: 1.7320 - accuracy: 0.3838
Epoch 3/10
366/366 [==============================] - 0s 917us/step - loss: 1.5519 - accuracy: 0.3792
Epoch 4/10
366/366 [==============================] - 0s 852us/step - loss: 1.4265 - accuracy: 0.4007
Epoch 5/10
366/366 [==============================] - 0s 841us/step - loss: 1.3885 - accuracy: 0.4029
Epoch 6/10
366/366 [==============================] - 0s 791us/step - loss: 1.3351 - accuracy: 0.4180
Epoch 7/10
366/366 [==============================] - 0s 843us/step - loss: 1.3198 - accuracy: 0.4224
Epoch 8/10
366/366 [==============================] - 0s 848us/step - loss: 1.3030 - accuracy: 0.4298
Epoch 9/10
366/366 [==============================] - 0s 876us/step - loss: 1.2877 - accuracy: 0.4298
Epoch 10/10
366/366 [==============================] - 0s 857us/step - loss: 1.2791 - 

366/366 [==============================] - 1s 1ms/step - loss: 11.0235 - accuracy: 0.2887
Epoch 2/10
366/366 [==============================] - 0s 757us/step - loss: 3.5134 - accuracy: 0.3460
Epoch 3/10
366/366 [==============================] - 0s 812us/step - loss: 2.1020 - accuracy: 0.3593
Epoch 4/10
366/366 [==============================] - 0s 931us/step - loss: 1.6223 - accuracy: 0.3973
Epoch 5/10
366/366 [==============================] - 0s 788us/step - loss: 1.4822 - accuracy: 0.4088
Epoch 6/10
366/366 [==============================] - 0s 786us/step - loss: 1.3968 - accuracy: 0.4192
Epoch 7/10
366/366 [==============================] - 0s 812us/step - loss: 1.3588 - accuracy: 0.4173
Epoch 8/10
366/366 [==============================] - 0s 777us/step - loss: 1.3396 - accuracy: 0.4296
Epoch 9/10
366/366 [==============================] - 0s 779us/step - loss: 1.2999 - accuracy: 0.4424
Epoch 10/10
366/366 [==============================] - 0s 799us/step - loss: 1.2882 - accuracy

366/366 [==============================] - 1s 1ms/step - loss: 11.8645 - accuracy: 0.2480
Epoch 2/20
366/366 [==============================] - 1s 2ms/step - loss: 2.8124 - accuracy: 0.3446
Epoch 3/20
366/366 [==============================] - 1s 2ms/step - loss: 1.9413 - accuracy: 0.3698
Epoch 4/20
366/366 [==============================] - 0s 821us/step - loss: 1.7269 - accuracy: 0.3935
Epoch 5/20
366/366 [==============================] - 0s 1ms/step - loss: 1.5624 - accuracy: 0.4088
Epoch 6/20
366/366 [==============================] - 0s 988us/step - loss: 1.4700 - accuracy: 0.4125
Epoch 7/20
366/366 [==============================] - 0s 723us/step - loss: 1.4200 - accuracy: 0.4175
Epoch 8/20
366/366 [==============================] - 0s 945us/step - loss: 1.3651 - accuracy: 0.4219
Epoch 9/20
366/366 [==============================] - 0s 951us/step - loss: 1.3364 - accuracy: 0.4288
Epoch 10/20
366/366 [==============================] - 0s 1ms/step - loss: 1.3170 - accuracy: 0.4303

366/366 [==============================] - 1s 1ms/step - loss: 5.9169 - accuracy: 0.2435
Epoch 2/20
366/366 [==============================] - 0s 1ms/step - loss: 1.8979 - accuracy: 0.3202
Epoch 3/20
366/366 [==============================] - 0s 980us/step - loss: 1.4906 - accuracy: 0.3917
Epoch 4/20
366/366 [==============================] - 0s 780us/step - loss: 1.3866 - accuracy: 0.4093
Epoch 5/20
366/366 [==============================] - 0s 780us/step - loss: 1.3467 - accuracy: 0.4127
Epoch 6/20
366/366 [==============================] - 0s 1ms/step - loss: 1.3212 - accuracy: 0.4264
Epoch 7/20
366/366 [==============================] - 0s 1ms/step - loss: 1.2986 - accuracy: 0.4428
Epoch 8/20
366/366 [==============================] - 0s 889us/step - loss: 1.2889 - accuracy: 0.4500
Epoch 9/20
366/366 [==============================] - 0s 841us/step - loss: 1.2814 - accuracy: 0.4519
Epoch 10/20
366/366 [==============================] - 0s 858us/step - loss: 1.2726 - accuracy: 0.448

...

366/366 [==============================] - 0s 1ms/step - loss: 1.2925 - accuracy: 0.4253
Epoch 11/20
366/366 [==============================] - 0s 1ms/step - loss: 1.2875 - accuracy: 0.4285
Epoch 12/20
366/366 [==============================] - 0s 1ms/step - loss: 1.2848 - accuracy: 0.4307
Epoch 13/20
366/366 [==============================] - 0s 1ms/step - loss: 1.2856 - accuracy: 0.4224
Epoch 14/20
366/366 [==============================] - 0s 1ms/step - loss: 1.2831 - accuracy: 0.4225
Epoch 15/20
366/366 [==============================] - 0s 1ms/step - loss: 1.2822 - accuracy: 0.4284
Epoch 16/20
366/366 [==============================] - 0s 1ms/step - loss: 1.2813 - accuracy: 0.4318
Epoch 17/20
366/366 [==============================] - 0s 1ms/step - loss: 1.2789 - accuracy: 0.4364
Epoch 18/20
366/366 [==============================] - 0s 1ms/step - loss: 1.2798 - accuracy: 0.4364
Epoch 19/20
366/366 [==============================] - 0s 1ms/step - loss: 1.2785 - accuracy: 0.4364
Ep

366/366 [==============================] - 0s 1ms/step - loss: 1.3051 - accuracy: 0.4367
Epoch 12/20
366/366 [==============================] - 0s 1ms/step - loss: 1.2966 - accuracy: 0.4395
Epoch 13/20
366/366 [==============================] - 0s 1ms/step - loss: 1.2977 - accuracy: 0.4383
Epoch 14/20
366/366 [==============================] - 0s 1ms/step - loss: 1.2897 - accuracy: 0.4326
Epoch 15/20
366/366 [==============================] - 0s 1ms/step - loss: 1.2815 - accuracy: 0.4415
Epoch 16/20
366/366 [==============================] - 0s 1ms/step - loss: 1.2773 - accuracy: 0.4395
Epoch 17/20
366/366 [==============================] - 0s 1ms/step - loss: 1.2728 - accuracy: 0.4453
Epoch 18/20
366/366 [==============================] - 0s 1ms/step - loss: 1.2723 - accuracy: 0.4410
Epoch 19/20
366/366 [==============================] - 0s 1ms/step - loss: 1.2688 - accuracy: 0.4446
Epoch 20/20
366/366 [==============================] - 0s 1ms/step - loss: 1.2655 - accuracy: 0.4446
Ep

366/366 [==============================] - 0s 1ms/step - loss: 1.2725 - accuracy: 0.4372
Epoch 13/20
366/366 [==============================] - 0s 1ms/step - loss: 1.2716 - accuracy: 0.4376
Epoch 14/20
366/366 [==============================] - 0s 1ms/step - loss: 1.2682 - accuracy: 0.4407
Epoch 15/20
366/366 [==============================] - 1s 1ms/step - loss: 1.2674 - accuracy: 0.4417
Epoch 16/20
366/366 [==============================] - 0s 1ms/step - loss: 1.2659 - accuracy: 0.4432
Epoch 17/20
366/366 [==============================] - 0s 1ms/step - loss: 1.2639 - accuracy: 0.4419
Epoch 18/20
366/366 [==============================] - 0s 1ms/step - loss: 1.2630 - accuracy: 0.4441
Epoch 19/20
366/366 [==============================] - 0s 1ms/step - loss: 1.2631 - accuracy: 0.4439
Epoch 20/20
366/366 [==============================] - 0s 1ms/step - loss: 1.2629 - accuracy: 0.4410
Epoch 1/20
138/138 [==============================] - 0s 1ms/step - loss: 2.9000 - accuracy: 0.1833
Epo

366/366 [==============================] - 1s 1ms/step - loss: 1.2786 - accuracy: 0.4338
Epoch 14/20
366/366 [==============================] - 0s 1ms/step - loss: 1.2763 - accuracy: 0.4313
Epoch 15/20
366/366 [==============================] - 1s 2ms/step - loss: 1.2770 - accuracy: 0.4366
Epoch 16/20
366/366 [==============================] - 0s 1ms/step - loss: 1.2763 - accuracy: 0.4396
Epoch 17/20
366/366 [==============================] - 0s 1ms/step - loss: 1.2729 - accuracy: 0.4379
Epoch 18/20
366/366 [==============================] - 0s 1ms/step - loss: 1.2699 - accuracy: 0.4374
Epoch 19/20
366/366 [==============================] - 0s 1ms/step - loss: 1.2631 - accuracy: 0.4398
Epoch 20/20
366/366 [==============================] - 0s 1ms/step - loss: 1.2596 - accuracy: 0.4439
Epoch 1/20
138/138 [==============================] - 0s 1ms/step - loss: 3.9303 - accuracy: 0.0011
Epoch 2/20
138/138 [==============================] - 0s 1ms/step - loss: 2.9215 - accuracy: 0.0011
Epoc

366/366 [==============================] - 0s 1ms/step - loss: 1.2272 - accuracy: 0.4716
Epoch 15/20
366/366 [==============================] - 0s 1ms/step - loss: 1.2161 - accuracy: 0.4708
Epoch 16/20
366/366 [==============================] - 0s 1ms/step - loss: 1.2022 - accuracy: 0.4850
Epoch 17/20
366/366 [==============================] - 0s 1ms/step - loss: 1.1909 - accuracy: 0.4844
Epoch 18/20
366/366 [==============================] - 0s 1ms/step - loss: 1.1768 - accuracy: 0.4923
Epoch 19/20
366/366 [==============================] - 0s 1ms/step - loss: 1.1671 - accuracy: 0.4947
Epoch 20/20
366/366 [==============================] - 0s 1ms/step - loss: 1.1625 - accuracy: 0.5021
Epoch 1/20
183/183 [==============================] - 0s 2ms/step - loss: 1.7089 - accuracy: 0.3866
Epoch 2/20
183/183 [==============================] - 0s 1ms/step - loss: 1.3152 - accuracy: 0.4684
Epoch 3/20
183/183 [==============================] - 0s 1ms/step - loss: 1.2711 - accuracy: 0.4815
Epoch

In [19]:
time_end = time.time()
print((time_end - time_begin)/60)

55.056820742289226
